In [1]:
import torch
from PIL import Image
import requests

from transformers import Blip2Processor, Blip2ForConditionalGeneration
import random

import pickle
import time
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
#os.environ['HF_HOME'] = '/var/scratch/ybi530/data'

device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
print(device)
img_path = '/var/scratch/ybi530/data/data/train2014/'

cpu


In [2]:
## generate prompt
df = pd.read_csv('/var/scratch/ybi530/result/no_intention.csv',index_col=None)
df.head(3)

,Unnamed: 0,image_url,intention,target_action,action1,action2,action3,action4,correct_action,prompt,result_no_intention
0,23,COCO_train2014_000000421305.jpg,If I feel adventurous,E.I will ride an elephant,A.I will go bird watching on an outdoor publi...,C.I will stand and watch the zebras,B.I will just watch my friend fly an animal kite,D.I will ride a horse like the man,E,What am I supposed to do? pick the best option...,E
1,29,COCO_train2014_000000272797.jpg,If I am feeling lazy,C.I will ride on an elephant's neck,D.I will go to the museum and look at the fra...,E.I will i will have an elephant painting me ...,B.I will stand close to the bull and take a p...,A.I will take them for a ride on a horse carr...,C,What am I supposed to do? pick the best option...,C
2,30,COCO_train2014_000000272797.jpg,If I if i feeling very joy,D.I will i will go to zoo,B.I will watch an elephant take off a mans ha...,A.I will give the child food to feed the elep...,E.I will have some fun in the park,C.I will take my kite out to the backyard,D,What am I supposed to do? pick the best option...,A


In [3]:
def concat_choice(row):
    use_columns = ['target_action', 'action1', 'action2',
       'action3', 'action4']
    choices = []
    
#     如何改成按顺序加ABCDE
    for i in use_columns:
        choices.append(row[i]+'.')
        choices = sorted(choices)
        result = ' '.join(choices)
    return result+' '
    
df['choices'] = df.apply(concat_choice, axis=1)

In [4]:
def concat_intention(row,prompt):
    result = prompt.replace('INTENTION', row['intention'])
    result = result.replace('CHOICES', row['choices'])
    return result

In [5]:
def create_prompts(df):
    prompt_list =["What am I supposed to do INTENTION?CHOICES",
"What am I supposed to do INTENTION? Pick the best option from the following ones:CHOICES",
"Question: What am I supposed to do INTENTION? Pick the best option from the following ones:CHOICES Answer:",
"Given this visual context, what am I supposed to do INTENTION? Pick the best option from the following ones:CHOICES",
"What am I supposed to do given this visual context, INTENTION? Pick the best option from the following ones:CHOICES",
"Suppose I am faced with this situation. What am I supposed to do INTENTION? Pick the best option from the following ones:CHOICES",
"What action will I perform INTENTION? Pick the best option from the following ones:CHOICES",
"Suppose I am faced with this situation. What actions should I perform INTENTION? Pick the best option from the following ones:CHOICES",
"Imagine I am in this visual scenario. What action should I perform INTENTION? Pick the best option from the following ones:CHOICES",
"What actions should I perform INTENTION? Considering both my feeling and the image, pick the best option from the following ones:,CHOICES",
"Considering both my feeling and the image, what will I do INTENTION? Pick the best option from the following ones:CHOICES",
"What will I do INTENTION? Pick the best option from the following ones:CHOICES",
"INTENTION, what's the best step to take? Select the optimal choice from the options provided.CHOICES",
"INTENTION, what's the most suitable action to choose? Select the best option from the following list.CHOICES",
"INTENTION, what's the recommended action to perform? Choose the best option from the following ones.CHOICES",
"Which action is most advisable INTENTION? Choose from the following options.CHOICES",
"What's the best step to follow INTENTION? Pick from the following options:CHOICES",
"INTENTION, what's the most appropriate decision to make? Select the right choice from the options provided:CHOICES",
"What's the recommended response INTENTION? Choose from the following options.CHOICES",
"What should I do INTENTION? Choose from the following options considering both my feeling and the image.CHOICES",
"Taking into account both my feeling and the image, what's the recommended step to take INTENTION? Pick from the following options.CHOICES",
"Considering equally the image and my feeling, what should be my action INTENTION? Choose the right action from the following list.CHOICES",
"With both the image and my feeling in mind, how should I react INTENTION? Select the best choice from the following options.CHOICES",
"With both the image and my feeling in mind, what action is most advisable INTENTION? Choose from the following options.CHOICES",
"When taking both the image and my feeling into consideration, what's the best step to follow INTENTION? Pick from the following options.CHOICES",
"INTENTION.CHOICES",
"INTENTION, what happens next?CHOICES",
"INTENTION, what will I do?CHOICES",
"What will I do INTENTION?CHOICES",
"How should I behave INTENTION?CHOICES "]
    for i, j in enumerate(prompt_list):
        df['prompt_'+str(i)] = df.apply(lambda x:concat_intention(x,j), axis=1)
       
    return df

In [6]:
df = create_prompts(df)

In [7]:
df.to_csv('result_Nov/df_with_prompts.csv')